In [ ]:
from typing import TypedDict,Annotated
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from langgraph.graph import StateGraph,START,END
from langchain_core.output_parsers import PydanticOutputParser
from operator import add

In [ ]:
class LLMState(TypedDict):
    essay_txt:str
    COT_feedback:str
    DOA_feedback:str
    Language_feedback:str
    Summarized:str
    individual_score:Annotated[list[int],add] # Use add reducer to accumulate scores
    Avg_Score:float

In [ ]:
load_dotenv()
llm=ChatCohere(model="command-r-plus-08-2024")

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

class Reviews(BaseModel):
    Text_Feedback: str = Field(description="Detailed feedback based on the criteria")
    Score: Literal[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] = Field(description="Score from 0-10 based on the criteria")

# Create structured output model
structured_model = llm.with_structured_output(schema=Reviews)

In [ ]:
def Clarity_of_Thought(state:LLMState)->dict:
    prompt=f"""
    Based on "Clarity of thoughts" of this essay just give me a detailed review of this essay and score it on the scale of 10 ,here is the essay:\n
    {state["essay_txt"]}
    """
    COT_feedback=structured_model.invoke(prompt)
    return {"COT_feedback":COT_feedback.Text_Feedback,"individual_score":[COT_feedback.Score]}


def Depth_of_Analysis(state:LLMState)->dict:
    prompt=f"""
    Based on "Depth of Anaysis" of this essay just give me a detailed review of this essay and score it on the scale of 10 ,here is the essay:\n
    {state["essay_txt"]}
    """
    DOA_feedback=structured_model.invoke(prompt)
    return {"DOA_feedback":DOA_feedback.Text_Feedback,"individual_score":[DOA_feedback.Score]}


def Language_feedback(state:LLMState)->dict:
    prompt=f"""
    Based on "Language" of this essay just give me a detailed review of this essay and score it on the scale of 10 ,here is the essay:\n
    {state["essay_txt"]}
    """
    L_feedback=structured_model.invoke(prompt)
    return {"Language_feedback":L_feedback.Text_Feedback,"individual_score":[L_feedback.Score]}

def Summary(state:LLMState)->dict:
    prompt=f"""
    Based on feedback on the basis of
    "Clarity of thoughts": {state['COT_feedback']}\n
    "Depth of Analysis"{state["DOA_feedback"]}\n
    "Language":{state["Language_feedback"]}\n
    of this essay:{state["essay_txt"]}
    just give me a detailed review of this essay and give the average of the scores of it form all of these three factors.
    And the scores are
    "Clarity of thoughts": {state["individual_score"][0]}\n
    "Depth of Analysis"{state["individual_score"][1]}\n
    "Language":{state["individual_score"][2]}\n
    """
    
    Summary_feedback=structured_model.invoke(prompt)
    avg_score=sum(state["individual_score"])/len(state["individual_score"])
    return {"Summarized":Summary_feedback.Text_Feedback,"Avg_Score":avg_score}

In [ ]:
graph=StateGraph(LLMState)

graph.add_node("Clarity_of_Thought",Clarity_of_Thought)
graph.add_node("Depth_of_Analysis",Depth_of_Analysis)
graph.add_node("Language_feedback",Language_feedback)
graph.add_node("Summary",Summary)

graph.add_edge(START,"Clarity_of_Thought")
graph.add_edge(START,"Depth_of_Analysis")
graph.add_edge(START,"Language_feedback")

graph.add_edge("Clarity_of_Thought","Summary")
graph.add_edge("Depth_of_Analysis","Summary")
graph.add_edge("Language_feedback","Summary")

graph.add_edge("Summary",END)

workflow=graph.compile()

In [ ]:
essay="""
Technology has become an essential part of modern life, influencing how we work, learn, and communicate. Over the past decade, digital tools have transformed traditional industries, making information more accessible and tasks more efficient. However, this rapid growth also brings challenges, such as privacy concerns and increased screen time.

Despite these issues, the benefits of technology continue to outweigh the drawbacks. It enables global collaboration, supports innovation, and offers countless opportunities for personal and professional growth. As technology evolves, society must learn to use it responsibly to maximize its positive impact.
"""

final_output=workflow.invoke({"essay_txt":essay})

In [ ]:
final_output["Summarized"]

"Your essay presents a coherent argument about technology's impact on society, but it could benefit from further development and specific examples to strengthen your points. The introduction is effective in capturing the essence of technology's role in our lives, but providing real-world instances would make your argument more compelling. For example, you could illustrate how technology has revolutionized communication by discussing the rise of social media and its impact on global connectivity. Or, you might explore the transformation of the education sector through online learning platforms, highlighting both the benefits and potential drawbacks."